This code combine all columns from the large ecommerce dataset, and save into a new csv file.

In [ ]:
#If opening in colab run this cell
!git clone https://github.com/jason2003120/DS_FinalProject.git
%cd DS_FinalProject/

In [1]:
# Trick to display plots inline with the rest of your notebook.
%matplotlib inline
import pandas as pd

In [2]:
########################################################################
# Load large dataset
########################################################################

# Define the base path to the 'large' directory containing the CSV files
base_path = './data/large/'  # Adjust this path as necessary
# Load each CSV file into a separate dataframe
customers_df = pd.read_csv(f'{base_path}CUSTOMERS.csv')
geo_location_df = pd.read_csv(f'{base_path}GEO_LOCATION.csv')
order_items_df = pd.read_csv(f'{base_path}ORDER_ITEMS.csv')
order_payments_df = pd.read_csv(f'{base_path}ORDER_PAYMENTS.csv')
order_review_ratings_df = pd.read_csv(f'{base_path}ORDER_REVIEW_RATINGS.csv')
orders_df = pd.read_csv(f'{base_path}ORDERS.csv')
products_df = pd.read_csv(f'{base_path}PRODUCTS.csv')

sellers_df = pd.read_csv(f'{base_path}SELLERS.csv')

# Optionally, print the first few rows of each dataframe to verify successful loading
print(customers_df.head())
print(geo_location_df.head())
print(order_items_df.head())
print(order_payments_df.head())
print(order_review_ratings_df.head())
print(orders_df.head())
print(products_df.head())
print(sellers_df.head())

                        customer_id                customer_unique_id  \
0  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1  18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   
2  4e7b3e00288586ebd08712fdd0374a03  060e732b5b29e8181a18229c7b0b2b5e   
3  b2b6027bc5c5109e529d4dc6358b12c3  259dac757896d24d7702b9acbbff3f3c   
4  4f2d8ab171c80ec8364f7c12e35b23ad  345ecd01c38d18a9036ed96c73b8d066   

   customer_zip_code_prefix customer_city  customer_state  
0                     14409      Adilabad  Andhra Pradesh  
1                      9790         Adoni  Andhra Pradesh  
2                      1151  Akkarampalle  Andhra Pradesh  
3                      8775   Akkayapalle  Andhra Pradesh  
4                     13056         Alwal  Andhra Pradesh  
   geolocation_zip_code_prefix  geolocation_lat  geolocation_lng  \
0                         1037       -23.545621       -46.639292   
1                         1046       -23.546081       -46.644820 

In [3]:
# Merge orders with customer details
orders_customers_df = pd.merge(orders_df, customers_df, on='customer_id', how='left')

# Merge order items to the orders and customers data
orders_customers_items_df = pd.merge(orders_customers_df, order_items_df, on='order_id', how='left')

# Merge product details
full_df = pd.merge(orders_customers_items_df, products_df, on='product_id', how='left')

# Merge payment details
full_df = pd.merge(full_df, order_payments_df, on='order_id', how='left')

# Merge review ratings
full_df = pd.merge(full_df, order_review_ratings_df, on='order_id', how='left')

# Merge seller details
full_df = pd.merge(full_df, sellers_df, on='seller_id', how='left')

# Example: Enriching customer city and state (assuming customer and seller share the same zip prefix)
# This is a simplification and might not directly apply if your dataset requires precise geo mapping.
full_df = pd.merge(full_df, geo_location_df, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left', suffixes=('', '_geo'))
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115707 entries, 0 to 115706
Data columns (total 42 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       115707 non-null  object 
 1   customer_id                    115707 non-null  object 
 2   order_status                   115707 non-null  object 
 3   order_purchase_timestamp       115707 non-null  object 
 4   order_approved_at              115707 non-null  object 
 5   order_delivered_carrier_date   115707 non-null  object 
 6   order_delivered_customer_date  115707 non-null  object 
 7   order_estimated_delivery_date  115707 non-null  object 
 8   customer_unique_id             115707 non-null  object 
 9   customer_zip_code_prefix       115707 non-null  int64  
 10  customer_city                  115707 non-null  object 
 11  customer_state                 115707 non-null  object 
 12  order_item_id                 

In [6]:
full_df.describe()

,customer_zip_code_prefix,order_item_id,price,freight_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,payment_sequential,payment_installments,payment_value,review_score,seller_zip_code_prefix,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng
count,115707.000000,115707.000000,115707.000000,115707.000000,114046.000000,114046.000000,114046.000000,114046.000000,114046.000000,114046.000000,114046.000000,115704.000000,115704.000000,115704.000000,115707.000000,115707.000000,115404.000000,115404.000000,115404.000000
mean,35081.100875,1.197067,119.911800,19.982046,48.801054,784.822317,2.206338,2108.507839,30.289927,16.605554,23.105396,1.091285,2.941039,171.815461,4.065130,24561.641595,35005.456275,-21.241334,-46.201453
std,29855.443121,0.701262,182.682279,15.719326,10.016649,650.602107,1.718005,3768.605007,16.158640,13.440159,11.740799,0.686784,2.776427,265.660487,1.359598,27692.583121,29842.171213,5.567641,4.048836
min,1003.000000,1.000000,0.850000,0.000000,5.000000,4.000000,1.000000,0.000000,7.000000,2.000000,6.000000,1.000000,0.000000,0.000000,1.000000,1001.000000,1003.000000,-36.605374,-72.666706
25%,11310.000000,1.000000,39.900000,13.080000,42.000000,345.000000,1.000000,300.000000,18.000000,8.000000,15.000000,1.000000,1.000000,60.850000,4.000000,6429.000000,11095.000000,-23.590774,-48.110471
50%,24320.000000,1.000000,74.900000,16.280000,52.000000,600.000000,1.000000,700.000000,25.000000,13.000000,20.000000,1.000000,2.000000,108.110000,5.000000,13660.000000,24240.000000,-22.929319,-46.632180
75%,58812.500000,1.000000,132.900000,21.160000,57.000000,983.000000,3.000000,1800.000000,38.000000,20.000000,30.000000,1.000000,4.000000,188.940000,5.000000,28805.000000,58390.000000,-20.198222,-43.632610
max,99980.000000,21.000000,6735.000000,409.680000,76.000000,3992.000000,20.000000,40425.000000,105.000000,105.000000,118.000000,26.000000,24.000000,13664.080000,5.000000,99730.000000,99980.000000,42.184003,-8.577855


In [5]:
full_df.to_csv('./data/dataset04052024.csv', index=False)  

In [9]:
from datetime import datetime
# Get the current date
current_date = datetime.now()

# Format the date as mmddyy
date_str = current_date.strftime('%m%d%y')

# Create the filename with the current date
filename = f'./data/dataset_{date_str}.csv'

# Save the DataFrame to a CSV file with the dynamically generated name
full_df.to_csv(filename, index=False)